In [1]:
#pip install requests html5lib bs4

#### Scrape des données de présentation de la FSTT:

Dans cette partie nous avons scraper les données de la page qui présente notre faculté et les mettres dans une base de données "DATA_FST" dans la collection "presentation_col"

In [2]:
#scrap Presentation data:
import requests
from bs4 import BeautifulSoup

URL = "https://fstt.ac.ma/Portail2023/presentation/#"
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
}

try:
    session = requests.Session()
    r = session.get(URL, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    div_content = soup.find('div', attrs={'class':"elementor-text-editor elementor-clearfix"})
    print(div_content.prettify())  # Afficher le contenu de la page, formaté de manière lisible
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


<div class="elementor-text-editor elementor-clearfix">
 <h4>
  <strong>
   I- PRESENTATION DE LA FST DE TANGER &amp; DIAGNOSIQUE
  </strong>
 </h4>
 <p>
  La Faculté des Sciences et Techniques de Tanger (FSTT), relevant de l’Université Abdelmalek Essaâdi a été créée en 1995. Elle fait partie des établissements de l’enseignement supérieur à accès régulé et a pour missions la formation initiale dans les domaines scientifiques et techniques, la formation continue ainsi que la recherche et le développement dans les domaines des sciences et techniques.
 </p>
 <p>
 </p>
 <p>
  <strong>
   <u>
    Formation Initiale
   </u>
  </strong>
 </p>
 <p>
  La FST de Tanger offre des cursus de formation qui préparent aux diplômes suivants :
 </p>
 <ul>
  <li>
   DEUST : Diplôme d’Etudes Universitaire en Sciences et Techniques (Bac +2).
  </li>
  <li>
   LST : Diplôme de Licence en Sciences et Techniques (Bac +3)
  </li>
  <li>
   MST : Diplôme de Master en Sciences et Techniques (Bac +5)
  </li>
  <li

In [11]:
#Structurer les donnes dans une base de données mongodb
from pymongo import MongoClient

# Vérifier si la requête a réussi
if r.status_code == 200:
    if div_content:
        # Connexion à MongoDB
        client = MongoClient('mongodb://localhost:27017/')
        db = client['DATA_FSTT']
        collection = db['presentation_col']

        # Extraire les titres et les paragraphes à partir du div spécifique
        elements = div_content.find_all(['strong', 'p', 'li'])

        # Variables pour stocker le titre courant et le texte courant
        current_title = None
        current_text = ""

        # Parcourir les éléments et structurer les données pour MongoDB
        for element in elements:
            if element.name == 'strong' or element.find('u'):
                if current_title and current_text:
                    # Insérer dans MongoDB
                    document = {
                        'titre': current_title,
                        'text': current_text.strip(),
                        'categorie': 'presentation',
                        'Questions possibles': ''
                    }
                    collection.insert_one(document)
                    # Réinitialiser le texte courant
                    current_text = ""
                current_title = element.get_text(strip=True)
            else:
                current_text += " " + element.get_text(strip=True)

        # Insérer le dernier élément dans MongoDB
        if current_title and current_text:
            document = {
                'titre': current_title,
                'text': current_text.strip(),
                'categorie': 'presentation',
                'Questions possibles': ''
            }
            collection.insert_one(document)

        print("Données insérées avec succès dans MongoDB.")
    else:
        print("Div spécifique non trouvé.")
else:
    print("La requête a échoué.")


Données insérées avec succès dans MongoDB.


***Note*** : J'ai apporté une modification à la base de données car le processus de collecte des données ne les divise pas correctement, ce qui crée une cellule supplémentaire nécessitant une suppression manuelle.

#### Scrape des données de Départements de la FSTT:

In [18]:
URL_1 = "https://fstt.ac.ma/Portail2023/les-departement/"
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
}

try:
    r = session.get(URL_1, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    div_content_dep = soup.find_all('div', attrs={'class':"elementor-cta__content"})
    for div in div_content_dep:
        print(div.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


<div class="elementor-cta__content">
 <h3 class="elementor-cta__title elementor-cta__content-item elementor-content-item">
  GÉNIE INFORMATIQUE
 </h3>
 <div class="elementor-cta__description elementor-cta__content-item elementor-content-item">
  Chef : Pr.EL BRAK Mohamed
  <br/>
  <b>
   Email :
  </b>
  melbrak@uae.ac.ma
 </div>
</div>

<div class="elementor-cta__content">
 <h3 class="elementor-cta__title elementor-cta__content-item elementor-content-item">
  GÉNIE CHIMIQUE
 </h3>
 <div class="elementor-cta__description elementor-cta__content-item elementor-content-item">
  Chef : Pr.CHABBI Mohamed
  <br/>
  <b>
   Email :
  </b>
  mchabbi@uae.ac.ma
 </div>
</div>

<div class="elementor-cta__content">
 <h3 class="elementor-cta__title elementor-cta__content-item elementor-content-item">
  SCIENCES DE LA TERRE
 </h3>
 <div class="elementor-cta__description elementor-cta__content-item elementor-content-item">
  Chef : Pr.BOULAASSAL Hakim
  <br/>
  <b>
   Email :
  </b>
  h.boulaassal@uae

In [28]:
import re
if r.status_code == 200:
    if div_content_dep:
        # Connexion à MongoDB
        client = MongoClient('mongodb://localhost:27017/')
        db = client['DATA_FSTT']
        collection = db['departement_col']
        # Regex pour extraire les emails
        email_regex = re.compile(r'[a-zA-Z]+(\.[a-zA-Z]+)?@uae\.ac\.ma')

        for div in div_content_dep:
            title = div.find('h3', class_='elementor-cta__title').get_text(strip=True)
            description = div.find('div', class_='elementor-cta__description').get_text(separator='\n').strip()
            
            # Extraction du chef
            chef = re.search(r'Chef\s*:\s*(.*)', description)
            chef = chef.group(1).strip() if chef else 'N/A'
            
            # Extraction de l'email avec regex
            email_match = email_regex.search(description)
            email = email_match.group(0) if email_match else 'N/A'
            
            departement_info = {
                "Departement": title,
                "Chef": chef,
                "Email": email,
                "categorie":"departement"
            }
            
            # Insérer les données dans la collection MongoDB
            collection.insert_one(departement_info)
            
            print(departement_info)
else:
    print("La requête a échoué.")

{'Departement': 'GÉNIE INFORMATIQUE', 'Chef': 'Pr.EL BRAK Mohamed', 'Email': 'melbrak@uae.ac.ma', 'categorie': 'departement', '_id': ObjectId('664c9ff4f085605b34f647a1')}
{'Departement': 'GÉNIE CHIMIQUE', 'Chef': 'Pr.CHABBI Mohamed', 'Email': 'mchabbi@uae.ac.ma', 'categorie': 'departement', '_id': ObjectId('664c9ff4f085605b34f647a2')}
{'Departement': 'SCIENCES DE LA TERRE', 'Chef': 'Pr.BOULAASSAL Hakim', 'Email': 'h.boulaassal@uae.ac.ma', 'categorie': 'departement', '_id': ObjectId('664c9ff4f085605b34f647a3')}
{'Departement': 'GÉNIE MÉCANIQUE', 'Chef': 'Pr.ELAYACHI Ilham', 'Email': 'i.elayachi@uae.ac.ma', 'categorie': 'departement', '_id': ObjectId('664c9ff4f085605b34f647a4')}
{'Departement': 'SCIENCES DE LA VIE', 'Chef': 'Pr.HASSANI ZERROUK Mounir', 'Email': 'mhassani@uae.ac.ma', 'categorie': 'departement', '_id': ObjectId('664c9ff4f085605b34f647a5')}
{'Departement': 'GÉNIE ELECTRIQUE', 'Chef': 'Pr.HADJ BARAKA Ibrahim', 'Email': 'ihadjbaraka@uae.ac.ma', 'categorie': 'departement', '_i

#### Scrape des données de Conseil et de commission de la FSTT:

###### #commission-suivi-du-budget

In [46]:
URL_2 = "https://fstt.ac.ma/Portail2023/commission-suivi-du-budget/#"

try:
    r = session.get(URL_2, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com = soup.find_all('td')
    #for tr in tr_content_com:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Commission d'Activités Culturelles et Sportives

In [49]:
URL_3 = "https://fstt.ac.ma/Portail2023/commission-activites-culturelles-et-sportives/#"

try:
    r = session.get(URL_3, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com3 = soup.find_all('td')
    #for tr in tr_content_com3:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Commission pédagogique:

In [50]:
URL_4 = "https://fstt.ac.ma/Portail2023/commission-pedagogique/"

try:
    r = session.get(URL_4, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com4 = soup.find_all('td')
    #for tr in tr_content_com4:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Commission de la Recherche Scientifique et de Coopération:

In [52]:
URL_5 = "https://fstt.ac.ma/Portail2023/commission-recherche-scientifique-cooperation/"

try:
    r = session.get(URL_5, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com5 = soup.find_all('td')
    #for tr in tr_content_com5:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Commission Scientifique:

In [54]:
URL_6 = "https://fstt.ac.ma/Portail2023/commission-scientifique/"

try:
    r = session.get(URL_6, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com6 = soup.find_all('td')
    #for tr in tr_content_com6:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Conseil de l’établissement:

In [57]:
URL_7 = "https://fstt.ac.ma/Portail2023/conseil-de-letablissement/"

try:
    r = session.get(URL_7, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    tr_content_com7 = soup.find_all('td')
    #for tr in tr_content_com7:
        #print(tr.prettify())
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


###### #Insertion des données scraper à la base de données:

In [43]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_ = tab[::2]
    resp_ = tab[1::2]
    
    for member, responsibility in zip(mem_, resp_):
        member_info = {
            "Membre": member,
            "Responsabilité": responsibility,
            "Categorie": "conseil_commission",
            "Commission": ["commission suivi du budget"]
        }
        collection.insert_one(member_info)
        print(member_info)  # Afficher les données insérées pour vérification


{'Membre': 'DIANI Mustapha', 'Responsabilité': 'Doyen', 'Categorie': 'conseil_commission', 'Commission': ['commission suivi du budget'], '_id': ObjectId('664cab2ef085605b34f647aa')}
{'Membre': 'JBILOU Mohammed', 'Responsabilité': 'Vice Doyen de la Formation', 'Categorie': 'conseil_commission', 'Commission': ['commission suivi du budget'], '_id': ObjectId('664cab2ef085605b34f647ab')}
{'Membre': 'OUARDOUZ Mustapha', 'Responsabilité': 'Vice Doyen de la Recherche et de la coopération', 'Categorie': 'conseil_commission', 'Commission': ['commission suivi du budget'], '_id': ObjectId('664cab2ef085605b34f647ac')}
{'Membre': 'AJDOUR Mounia', 'Responsabilité': 'Chef du département de Physique', 'Categorie': 'conseil_commission', 'Commission': ['commission suivi du budget'], '_id': ObjectId('664cab2ef085605b34f647ad')}
{'Membre': 'CHABBI Mohamed', 'Responsabilité': 'Chef du département de Génie Chimique', 'Categorie': 'conseil_commission', 'Commission': ['commission suivi du budget'], '_id': Obje

In [48]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com3:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_3 = tab[::2]
    resp_3 = tab[1::2]
    
    for member, responsibility in zip(mem_3, resp_3):
        existing_member = collection.find_one({"Membre": member})

        if existing_member:
            collection.update_one(
                {"Membre": member},
                {"$addToSet": {"Commission": "Commission d'Activités Culturelles et Sportives"}}
            )
            print(f"Updated commissions for {member}")
        else:
            # Nouveau membre, ajouter à la collection
            member_info = {
                "Membre": member,
                "Responsabilité": responsibility,
                "Categorie": "conseil_commission",
                "Commission": ["Commission d'Activités Culturelles et Sportives"]
            }
            collection.insert_one(member_info)
            print(member_info)  # Afficher les données insérées pour vérification


Updated commissions for DIANI Mustapha
Updated commissions for JBILOU Mohammed
Updated commissions for OUARDOUZ Mustapha
Updated commissions for AJDOUR Mounia
Updated commissions for CHABBI Mohamed
Updated commissions for EL HALIMI Rachid
Updated commissions for EL BRAK Mohamed
Updated commissions for BOULAASSAL Hakim
Updated commissions for HADJ BARAKA Ibrahim
Updated commissions for ELAYACHI Ilham
Updated commissions for OUCHEN Mohamed
Updated commissions for HASSANI ZERROUK Mounir
{'Membre': 'JAYED Youssef', 'Responsabilité': 'Représentant des étudiants', 'Categorie': 'conseil_commission', 'Commission': ["Commission d'Activités Culturelles et Sportives"], '_id': ObjectId('664cafd8f085605b34f647c1')}


In [51]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com4:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_4 = tab[::2]
    resp_4 = tab[1::2]
    
    for member, responsibility in zip(mem_4, resp_4):
        existing_member = collection.find_one({"Membre": member})

        if existing_member:
            collection.update_one(
                {"Membre": member},
                {"$addToSet": {"Commission": "Commission  pédagogique"}}
            )
            print(f"Updated commissions for {member}")
        else:
            # Nouveau membre, ajouter à la collection
            member_info = {
                "Membre": member,
                "Responsabilité": responsibility,
                "Categorie": "conseil_commission",
                "Commission": ["Commission  pédagogique"]
            }
            collection.insert_one(member_info)
            print(member_info)  # Afficher les données insérées pour vérification


Updated commissions for DIANI Mustapha
Updated commissions for JBILOU Mohammed
Updated commissions for OUARDOUZ Mustapha
Updated commissions for AJDOUR Mounia
Updated commissions for CHABBI Mohamed
Updated commissions for EL HALIMI Rachid
Updated commissions for BOULAASSAL Hakim
Updated commissions for HADJ BARAKA Ibrahim
Updated commissions for ELAYACHI Ilham
Updated commissions for OUCHEN Mohamed
Updated commissions for HASSANI ZERROUK Mounir
Updated commissions for EL BRAK Mohamed
Updated commissions for BOUASSAB Abderrahman
Updated commissions for EL METOUI Mustapha
Updated commissions for EN-NAIMI El Mokhtar
Updated commissions for DERFOUFI Soufiane
Updated commissions for EL MERZGUIOUI Mhamed
Updated commissions for JEBARI HASSANI Khalid
Updated commissions for ZANOUNI Mohamed
Updated commissions for BOUZID Saida
Updated commissions for FIKRI BENBRAHIM Chahinaze
Updated commissions for KHALI ISSA Sanae
Updated commissions for JAYED Youssef


In [53]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com5:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_5 = tab[::2]
    resp_5 = tab[1::2]
    
    for member, responsibility in zip(mem_5, resp_5):
        existing_member = collection.find_one({"Membre": member})

        if existing_member:
            collection.update_one(
                {"Membre": member},
                {"$addToSet": {"Commission": "Commission de la Recherche Scientifique et de Coopération"}}
            )
            print(f"Updated commissions for {member}")
        else:
            # Nouveau membre, ajouter à la collection
            member_info = {
                "Membre": member,
                "Responsabilité": responsibility,
                "Categorie": "conseil_commission",
                "Commission": ["Commission de la Recherche Scientifique et de Coopération"]
            }
            collection.insert_one(member_info)
            print(member_info)  # Afficher les données insérées pour vérification


Updated commissions for DIANI Mustapha
Updated commissions for JBILOU Mohammed
Updated commissions for OUARDOUZ Mustapha
Updated commissions for AJDOUR Mounia
Updated commissions for CHABBI Mohamed
Updated commissions for EL HALIMI Rachid
Updated commissions for BOULAASSAL Hakim
Updated commissions for HADJ BARAKA Ibrahim
Updated commissions for ELAYACHI Ilham
Updated commissions for OUCHEN Mohamed
Updated commissions for HASSANI ZERROUK Mounir
Updated commissions for EL BRAK Mohamed
Updated commissions for BOUASSAB Abderrahman
Updated commissions for EL METOUI Mustapha
Updated commissions for EN-NAIMI El Mokhtar
Updated commissions for DERFOUFI Soufiane
Updated commissions for EL MERZGUIOUI Mhamed
Updated commissions for JEBARI HASSANI Khalid
Updated commissions for ZANOUNI Mohamed
Updated commissions for BOUZID Saida
Updated commissions for FIKRI BENBRAHIM Chahinaze
Updated commissions for KHALI ISSA Sanae
Updated commissions for JAYED Youssef


In [55]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com6:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_6 = tab[::2]
    resp_6 = tab[1::2]
    
    for member, responsibility in zip(mem_6, resp_6):
        existing_member = collection.find_one({"Membre": member})

        if existing_member:
            collection.update_one(
                {"Membre": member},
                {"$addToSet": {"Commission": "Commission Scientifique"}}
            )
            print(f"Updated commissions for {member}")
        else:
            # Nouveau membre, ajouter à la collection
            member_info = {
                "Membre": member,
                "Responsabilité": responsibility,
                "Categorie": "conseil_commission",
                "Commission": ["Commission Scientifique"]
            }
            collection.insert_one(member_info)
            print(member_info)  # Afficher les données insérées pour vérification

Updated commissions for DIANI Mustapha
Updated commissions for OUARDOUZ Mustapha
Updated commissions for JBILOU Mohammed
{'Membre': 'ARHRIB Abdesslam', 'Responsabilité': 'P.E.S', 'Categorie': 'conseil_commission', 'Commission': ['Commission Scientifique'], '_id': ObjectId('664cb3fdf085605b34f647c2')}
{'Membre': 'EL HARZLI Mohamed', 'Responsabilité': 'P.E.S', 'Categorie': 'conseil_commission', 'Commission': ['Commission Scientifique'], '_id': ObjectId('664cb3fdf085605b34f647c3')}
Updated commissions for EN-NAIMI El Mokhtar
{'Membre': 'ROSSI Abdelhamid', 'Responsabilité': 'P.E.S', 'Categorie': 'conseil_commission', 'Commission': ['Commission Scientifique'], '_id': ObjectId('664cb3fdf085605b34f647c4')}
{'Membre': 'AMHARREF Mina', 'Responsabilité': 'P.E.S (Désignée)', 'Categorie': 'conseil_commission', 'Commission': ['Commission Scientifique'], '_id': ObjectId('664cb3fdf085605b34f647c5')}
{'Membre': 'CHAFIK Tarik', 'Responsabilité': 'P.E.S (Désigné)', 'Categorie': 'conseil_commission', 'Co

In [58]:
if r.status_code == 200:
    collection = db['commission_col']
    tab = []
    
    for td in tr_content_com7:
        div_content = td.find('div', class_='td-content')
        if div_content:
            tab.append(div_content.get_text().strip())

    mem_7 = tab[::2]
    resp_7 = tab[1::2]
    
    for member, responsibility in zip(mem_7, resp_7):
        existing_member = collection.find_one({"Membre": member})

        if existing_member:
            collection.update_one(
                {"Membre": member},
                {"$addToSet": {"Commission": "Membre de conseil de l’établissement"}}
            )
            print(f"Updated commissions for {member}")
        else:
            # Nouveau membre, ajouter à la collection
            member_info = {
                "Membre": member,
                "Responsabilité": responsibility,
                "Categorie": "conseil_commission",
                "Commission": ["Membre de conseil de l’établissement"]
            }
            collection.insert_one(member_info)
            print(member_info)  # Afficher les données insérées pour vérification

Updated commissions for DIANI Mustapha
Updated commissions for JBILOU Mohammed
Updated commissions for OUARDOUZ Mustapha
{'Membre': 'BIHRI Hassan', 'Responsabilité': 'SG chargé du secrétariat du conseil', 'Categorie': 'conseil_commission', 'Commission': ['Membre de conseil de l’établissement'], '_id': ObjectId('664cb501f085605b34f647c7')}
Updated commissions for AJDOUR Mounia
Updated commissions for CHABBI Mohamed
Updated commissions for EL HALIMI Rachid
Updated commissions for EL BRAK Mohamed
Updated commissions for BOULAASSAL Hakim
Updated commissions for HADJ BARAKA Ibrahim
Updated commissions for ELAYACHI Ilham
Updated commissions for OUCHEN Mohamed
Updated commissions for HASSANI ZERROUK Mounir
{'Membre': 'AOUNI Abdesamad', 'Responsabilité': 'Représentant des P.E.S', 'Categorie': 'conseil_commission', 'Commission': ['Membre de conseil de l’établissement'], '_id': ObjectId('664cb501f085605b34f647c8')}
Updated commissions for BOUASSAB Abderrahman
Updated commissions for EL METOUI Mu

#### Scrape des données de Formation initial de la FSTT:

##### #DEUST:

##### #BCG:

In [85]:
URL_8 = "https://fstt.ac.ma/Portail2023/biologie-chimie-geologie/"

data_bcg={"filere":"BIOLOGIE-CHIMIE-GEOLOGIE (BCG)",
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_8, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_8 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_8 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_8 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    
    if div_content_8:
        p_content = div_content_8.find_all('p')
        for p in p_content:
            data_bcg["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_8:
        text_content=div_coord_8.get_text().strip()
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_bcg["coordinateur"] = match.group("name").strip()
            data_bcg["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_8:
        prog_content=div_prog_8.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_bcg["programme"] = semestre_dict
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


In [86]:
data_bcg

{'filere': 'BIOLOGIE-CHIMIE-GEOLOGIE (BCG)',
 'objectif': "Le tronc commun BCG est un cursus qui comporte un ensemble de modules pris dans plusieurs champs disciplinaires et ayant pour objectif de faire acquérir des connaissances, des aptitudes et des compétences. Donner à l’étudiant les outils méthodologique pour qu’il puisse organiser son travail et développer un esprit de synthèse, de rigueur et d’initiative.L’objectif de cette formation et de donner à l’étudiant les possibilités d'orienter sa carrière vers les licences de chimie, de biologie et de géologie.Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs de notre établissement ainsi qu’au concours nationaux ou internationaux des écoles d’ingénieurs.",
 'coordinateur': 'Pr.Chouaibi Nour Eddine',
 'email': 'n.chouaibi@uae.ac.ma',
 'programme': {'Semestre 1': ['Biologie cellulaire',
   'Optique et Radioactivité',
   'Cosmologie & Géodynamique interne',
   'Structure de la matière'

##### #GE-GM:

In [87]:
URL_9 = "https://fstt.ac.ma/Portail2023/genie-electrique-genie-mecanique/"

data_gegm={"filere":"GÉNIE ELECTRIQUE – GÉNIE MÉCANIQUE (GE-GM)",
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_9, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_9 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_9 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_9 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    
    if div_content_9:
        p_content = div_content_9.find_all('p')
        for p in p_content:
            data_gegm["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_9:
        text_content=div_coord_9.get_text().strip()
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_gegm["coordinateur"] = match.group("name").strip()
            data_gegm["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_9:
        prog_content=div_prog_9.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_gegm["programme"] = semestre_dict
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_gegm

{'filere': 'GÉNIE ELECTRIQUE – GÉNIE MÉCANIQUE (GE-GM)',
 'objectif': 'L’objectif du tronc commun GE/GM est de donner à l’étudiant une base solide en matières scientifiques (physique, chimie et mathématiques etc…) et techniques (Electrotechnique, Automatique, mécanique et instrumentation et métrologie) qui vont lui permet de continuer ses études en licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter à d’autres licences des autres troncs communs comme MIP et MIPC.Également, ce tronc commun permet à l’étudiant de postuler à des concours nationaux ou internationaux des écoles d’ingénieurs.',
 'coordinateur': 'Pr.Jbilou Mohammed',
 'email': 'mjbilou@uae.ac.ma',
 'programme': {'Semestre 1': ['Circuits électriques et électroniques',
   'Electricité',
   'Analyse 1 : Fonction d’une variable réelle',
   'Algèbre 1 : Polynômes et espaces vectoriels',
   'Algorithmique et Programmation 1',
   'Langues et Communication -LC1'],
  'Semestre 2': ['Thermodynamique',
  

##### #MIP:

In [88]:
URL_10 = "https://fstt.ac.ma/Portail2023/mathematique-informatique-physique/"

data_mip={"filere":"MATHÉMATIQUE-INFORMATIQUE-PHYSIQUE (MIP)",
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_10, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_10 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_10 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_10 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    
    if div_content_10:
        p_content = div_content_10.find_all('p')
        for p in p_content:
            data_mip["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_10:
        text_content=div_coord_10.get_text().strip()
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_mip["coordinateur"] = match.group("name").strip()
            data_mip["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_10:
        prog_content=div_prog_10.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_mip["programme"] = semestre_dict
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_mip

{'filere': 'MATHÉMATIQUE-INFORMATIQUE-PHYSIQUE (MIP)',
 'objectif': 'L’objectif du tronc commun MIP est de donner à l’étudiant une base solide dans les matières scientifiques (physique, chimie et mathématiques etc…) qui vont lui permettre de continuer ses études dans les semestres S5 et S6 des cycles licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter vers d’autres troncs communs comme MIPC et GE/GM.Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs dispensés dans notre établissement et aussi à des concours nationaux ou internationaux des écoles d’ingénieurs.',
 'coordinateur': 'Pr.Jbilou Mohammed',
 'email': 'mjbilou@uae.ac.ma',
 'programme': {'Semestre 1': ['Circuits électriques et électroniques',
   'Electricité',
   'Analyse 1 : Fonction d’une variable réelle',
   'Algèbre 1 : Polynômes et espaces vectoriels',
   'Algorithmique et Programmation 1',
   'Langues et Communication -LC1'],
  'Semestre 2

##### #MIPC:

In [89]:
URL_11 = "https://fstt.ac.ma/Portail2023/mathematiques-informatique-physique-chimie/"

data_mipc={"filere":"MATHÉMATIQUES-INFORMATIQUE-PHYSIQUE-CHIMIE (MIPC)",
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_11, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_11 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_11 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_11 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    
    if div_content_11:
        p_content = div_content_11.find_all('p')
        for p in p_content:
            data_mipc["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_11:
        text_content=div_coord_11.get_text().strip()
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_mipc["coordinateur"] = match.group("name").strip()
            data_mipc["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_11:
        prog_content=div_prog_11.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_mipc["programme"] = semestre_dict
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_mipc

{'filere': 'MATHÉMATIQUES-INFORMATIQUE-PHYSIQUE-CHIMIE (MIPC)',
 'objectif': 'L’objectif du tronc commun MIPC est de donner à l’étudiant une base solide dans les matières scientifiques (physique, chimie et mathématiques etc…) qui vont lui permettre de continuer ses études dans les semestres S5 et S6 des cycles licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter vers d’autres troncs communs comme MIP et GE/GM.Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs dispensés dans notre établissement et aussi à des concours nationaux ou internationaux des écoles d’ingénieurs.',
 'coordinateur': 'Pr.Jbilou Mohammed',
 'email': 'mjbilou@uae.ac.ma',
 'programme': {'Semestre 1': ['Circuits électriques et électroniques',
   'Electricité',
   'Analyse 1 : Fonction d’une variable réelle',
   'Algèbre 1 : Polynômes et espaces vectoriels',
   'Algorithmique et Programmation 1',
   'Langues et Communication -LC1'],
  'S

In [98]:
db_formation_init = client['FORMATION_INITIAL']
collection_deust = db_formation_init['deust_col']
collection_deust.insert_one(data_bcg)
collection_deust.insert_one(data_gegm)
collection_deust.insert_one(data_mip)
collection_deust.insert_one(data_mipc)

InsertOneResult(ObjectId('664d16ccf085605b34f647d3'), acknowledged=True)

##### #LST:

##### #LST : Analytique des données

In [109]:
URL_12 = "https://fstt.ac.ma/Portail2023/lst-analytique-des-donnees/"

data_lstad={"filere":"LST : Analytique des données",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_12, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_12 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_12 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_12 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_12 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_12:
        p_content = div_content_12.find_all('p')
        for p in p_content:
            data_lstad["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_12:
        text_content=div_coord_12.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstad["coordinateur"] = match.group("name").strip()
            data_lstad["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_12:
        prog_content=div_prog_12.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstad["programme"] = semestre_dict

    if div_deb_12:
        deb_content=div_deb_12.get_text(separator=',').strip()
        #print(deb_content)
        data_lstad["debouche"]=deb_content
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstad

{'filere': 'LST : Analytique des données',
 'objectif': "La Licence Science et Techniques en analytique des données permet aux étudiants de doter de compétences en matière d'outils informatiques, des\ntechniques et des méthodes statistiques pour permettre d’organiser, de synthétiser et de traduire efficacement les données métier d’une\norganisation. L'étudiant doit être en mesure d'apporter un appui analytique à la conduite d'exploration et à l'analyse complexe de données.",
 'coordinateur': 'Pr.BAIDA Ouafae',
 'email': 'wbaida@uae.ac.ma',
 'debouche': ',Masters en sciences de données: fouille de données, business analytiques, blockchain,,Masters orientés e-Technologies: e-Business, e-Administration et e-Logistique,Formations d’Ingénieurs dans une école d’ingénieurs à l’issue de la deuxième ou de la troisième année de licence,Data scientist,Technicien supérieur en SGBD R : installation, configuration et administration des SGBD,WebMaster et développeur de sites web dynamiques,Intégratio

In [116]:
URL_13 = "https://fstt.ac.ma/Portail2023/lst-biotechnologies/"

data_lstbio={"filere":"LST: Biotechnologies",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_13, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_13 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_13 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_13 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_13 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_13:
        p_content = div_content_13.find_all('p')
        for p in p_content:
            data_lstbio["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_13:
        text_content=div_coord_13.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstbio["coordinateur"] = match.group("name").strip()
            data_lstbio["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_13:
        prog_content=div_prog_13.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstbio["programme"] = semestre_dict

    if div_deb_13:
        deb_content=div_deb_13.get_text(separator=',').strip()
        #print(deb_content)
        data_lstbio["debouche"]=deb_content.replace("\n,"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstbio

{'filere': 'LST: Biotechnologies',
 'objectif': 'L’objectif de la licence Biotechnologie consiste à former des étudiants dans les différents champs disciplinaires de la biologie moderne et de permettre aux étudiants d’acquérir de solides connaissances scientifiques pour une réelle compréhension du Vivant. L’enseignement est conçu pour allier concepts fondamentaux et apprentissage des outils méthodologiques. Il est également conçu pour étendre les compétences au-delà du domaine de la biologie, en particulier vers le monde économique. Ce parcours de licence vise donc à :',
 'coordinateur': 'Pr.Bouayad Noureddin',
 'email': 'nbouayad@uae.ac.ma',
 'debouche': "Les métiers auxquels peuvent prétendre les diplômés de la Licence Biotechnologie sont :  Les métiers de l'enseignement primaire, secondaire., Les métiers de la recherche appliquée et du développement dans les secteurs des biotechnologies, de l'agro-\xa0 alimentaire, de l'industrie pharmaceutique,..., Les métiers de la vente (délégués

In [119]:
URL_14 = "https://fstt.ac.ma/Portail2023/lst-design-industriel-et-productique/"

data_lstdip={"filere":"LST: Design Industriel et Productique",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_14, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_14 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_14 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_14 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_14 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_14:
        p_content = div_content_14.find_all('p')
        for p in p_content:
            data_lstdip["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_14:
        text_content=div_coord_14.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstdip["coordinateur"] = match.group("name").strip()
            data_lstdip["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_14:
        prog_content=div_prog_14.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstdip["programme"] = semestre_dict

    if div_deb_14:
        deb_content=div_deb_14.get_text().strip()
        #print(deb_content)
        data_lstdip["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstdip

{'filere': 'LST: Design Industriel et Productique',
 'objectif': 'Le Lauréat de Licence en Design et Productique en Génie Mécanique, à travers sa formation technique et scientifique, sera en mesure de suivre et d’intervenir dans toutes les étapes de Design, de réalisation, de Contrôle et de suivi du produit tout au long de son cycle de vie.',
 'coordinateur': 'Pr.Elmesbahi Abdelilah',
 'email': 'a.mesbahi@uae.ac.ma',
 'debouche': 'Planification et ordonnancement des opérations de fabrication ; Participation à l’élaboration d’un cahier de charge de produit mécanique ; Gestion de la production mécanique ; Maintenance des équipements mécaniques (Roulements, embrayages, pompes, …) ; Contrôle de la conformité des produits dans les Laboratoires et Département qualité : Spécification géométriques         et d’état de surface, Essais Mécaniques, Traitements thermiques des matériaux ;  Intégration des Services Techniques d’Achat de vente et après-vente.',
 'programme': {'Semestre 5': ['Choix de

In [120]:
URL_15 = "https://fstt.ac.ma/Portail2023/lst-energies-renouvelables/"

data_lster={"filere":"LST: Energies Renouvelables",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_15, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_15 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_15 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_15 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_15 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_15:
        p_content = div_content_15.find_all('p')
        for p in p_content:
            data_lster["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_15:
        text_content=div_coord_15.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lster["coordinateur"] = match.group("name").strip()
            data_lster["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_15:
        prog_content=div_prog_15.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lster["programme"] = semestre_dict

    if div_deb_15:
        deb_content=div_deb_15.get_text().strip()
        #print(deb_content)
        data_lster["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lster

{'filere': 'LST: Energies Renouvelables',
 'objectif': 'Approfondir les connaissances scientifiques et techniques en énergies renouvelables. Ce domaine inclut l’énergie solaire, éolienne, géothermique, hydraulique, thermoélectrique, L’énergie de la biomasse, le stockage de l’énergie, l’efficacité énergétique.Elle permettra aux étudiants de pouvoir créer leur entreprise pour l’installation et l’entretien des mini installations photovoltaïques, éolienne, donner du conseil au niveau EnR, ou intégrer des cycles d’ingénieurs ou de Master spécialisés.',
 'coordinateur': 'Pr.Ahachad Mohammed',
 'email': 'mahachad@uae.ac.ma',
 'debouche': 'Lors de cette formation, les étudiants auront à étudier les différentes formes de production des énergies renouvelables (EnR). Ils apprendront les fondements scientifiques et techniques sur l’énergie solaire (photovoltaïque, thermique), éolien, hydraulique, géothermique, l’énergie de biomasse et déchets, le stockage d’énergie, et l’efficacité énergétique.',


In [122]:
URL_16 = "https://fstt.ac.ma/Portail2023/lst-genie-civil/"

data_lstgc={"filere":"LST: Génie Civil",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_16, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_16 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_16 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_16 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_16 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_16:
        p_content = div_content_16.find_all('p')
        for p in p_content:
            data_lstgc["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_16:
        text_content=div_coord_16.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstgc["coordinateur"] = match.group("name").strip()
            data_lstgc["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_16:
        prog_content=div_prog_16.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstgc["programme"] = semestre_dict

    if div_deb_16:
        deb_content=div_deb_16.get_text().strip()
        #print(deb_content)
        data_lstgc["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstgc

{'filere': 'LST: Génie Civil',
 'objectif': '',
 'coordinateur': 'Pr.MABSSOUT Mokhtar',
 'email': 'mmabssout@uae.ac.ma',
 'debouche': 'La licence Génie Civil conduit vers un Master ou une formation d’ingénieur. Elle permet également\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 une insertion professionnelle, grâce aux différentes compétences acquises durant ce parcours.Cette insertion peut être dans des secteurs d’activités, comme des bureaux d’études ‘en Génie Civil, en environnement, en aménagement… etc), des entreprises du secteur du bâtiment et des travaux publics, des cabinets de maîtrise d’œuvre, des services de maîtrise d’œuvre, des services d’ouvrages ou de bureaux de contrôle technique.',
 'programme': {'Semestre 5': ['Mécanique des milieux continus et Outils Techniques en Génie Civil',
   'Résistance des matériaux',
   'Béton armé',
   'Mécanique des sols et notions de base de la géotechnique',
   'Matériaux de construction',
   'Méthodes numériques 

In [125]:
URL_17 = "https://fstt.ac.ma/Portail2023/lst-genie-industriel/"

data_lstgi={"filere":"LST : Génie Industriel",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_17, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_17 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_17 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_17 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_17 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_17:
        p_content = div_content_17.find_all('p')
        for p in p_content:
            data_lstgi["objectif"]+=p.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_17:
        text_content=div_coord_17.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstgi["coordinateur"] = match.group("name").strip()
            data_lstgi["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_17:
        prog_content=div_prog_17.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstgi["programme"] = semestre_dict

    if div_deb_17:
        deb_content=div_deb_17.get_text().strip()
        #print(deb_content)
        data_lstgi["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstgi

{'filere': 'LST : Génie Industriel',
 'objectif': 'Cette formation pluridisciplinaire garantit l’ouverture d’esprit nécessaire pour appréhender et dominer les problèmes concernant un projet industriel dont l’aspect technique ne constitue qu’une partie parmi d’autres.L’étudiant disposant d’une licence Sciences et techniques Génie Industriel pourra poursuivre ses études supérieures dans les différentes grandes écoles ou bien en Master Science et Techniques. Il pourra aussi intégrer le tissu industriel essentiellement les PME/PMI.',
 'coordinateur': 'Pr.Hassan El M’rabet',
 'email': 'helmramet@uae.ac.ma',
 'debouche': 'Métiers de l’ingénieur : Production ; Méthodes ; Maintenance ; Management de la qualité. Poursuite des études en Master ou en cycle d’ingénieur.',
 'programme': {'Semestre 5': ['Machines Hydrauliques',
   'Gestion de production',
   'Machines Thermiques',
   'Gestion de la qualité',
   'Gestion de la maintenance et sureté de fonctionnement',
   'Matériaux et RDM'],
  'Semes

In [127]:
URL_18 = "https://fstt.ac.ma/Portail2023/lst-genie-informatique/"

data_lstginfo={"filere":"LST : Génie Informatique",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_18, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_18 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_18 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_18 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_18 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_18:
        p_content = div_content_18.find_all('li')
        for li in p_content:
            data_lstginfo["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_18:
        text_content=div_coord_18.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstginfo["coordinateur"] = match.group("name").strip()
            data_lstginfo["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_18:
        prog_content=div_prog_18.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstginfo["programme"] = semestre_dict

    if div_deb_18:
        deb_content=div_deb_18.get_text().strip()
        #print(deb_content)
        data_lstginfo["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstginfo

{'filere': 'LST : Génie Informatique',
 'objectif': 'Avoir une culture de base scientifique\xa0;Acquérir une base solide dans les axes majeurs et fondamentaux de la discipline informatique : Algorithmique, Programmation, Bases de Données, Technologies Web, Systèmes d’Exploitation et Réseaux Informatiques, ... ;Développer un savoir-faire technique en informatique\xa0: technologie objet, informatique distribuée, architectures client-serveur et n-tiers, applications hétérogènes, …\xa0;',
 'coordinateur': 'Pr.Boudhir Anouar Abdelhakim',
 'email': 'aboudhir@uae.ac.ma',
 'debouche': 'Technicien supérieur en développement d’application en C++ et JAVA ; Technicien supérieur en réseaux locaux ; Technicien supérieur en SGBD-R : installation, configuration et administration des SGBD ; WebMaster et développeur des sites web dynamiques, ... Les orientations potentielles de poursuite des études sont de type : Masters à dominance informatique, Masters orientés e-Technologies, Masters à dominance auto

In [128]:
URL_19 = "https://fstt.ac.ma/Portail2023/lst-8-genie-des-procedes/"

data_lstgpro={"filere":"LST : Génie des Procédés",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_19, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_19 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_19 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_19 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_19 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_19:
        p_content = div_content_19.find_all('p')
        for li in p_content:
            data_lstgpro["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_19:
        text_content=div_coord_19.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstgpro["coordinateur"] = match.group("name").strip()
            data_lstgpro["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_19:
        prog_content=div_prog_19.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstgpro["programme"] = semestre_dict

    if div_deb_19:
        deb_content=div_deb_19.get_text().strip()
        #print(deb_content)
        data_lstgpro["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstgpro

{'filere': 'LST : Génie des Procédés',
 'objectif': 'Formation visant à donner aux étudiants des bases solides en chimie, physique, mathématiques, informatique et de gestion des procédés industriels. Les lauréats de la formation seront capables de conduire des ateliers de fabrication continue et séquentielle, de manager une équipe, de connaître et de prendre en compte les contraintes de sécurité, d’environnement et de l’organisation scientifique du travail.',
 'coordinateur': 'Pr.AZAAR Khalid',
 'email': 'k.azaar@uae.ac.ma',
 'debouche': 'Mettre l’étudiant au cœur des problèmes technologique réel qu’il devra résoudre.Le futur lauréat doit convaincre le secteur industriel et laboratoires de ses capacités scientifiques, multidisciplinaires, décisionnelles et organisationnelles.',
 'programme': {'Semestre 5': ['Calculs des réacteurs',
   'Bilan Matière et Énergie',
   'Opérations Unitaires',
   'Mécanique des fluides et Hydraulique',
   'Optimisation des Procédés Industrielle',
   'Modéli

In [129]:
URL_20 = "https://fstt.ac.ma/Portail2023/lst-mathematiques-et-applications/"

data_lstma={"filere":"LST: Mathématiques et Applications",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_20, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_20 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_20 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_20 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_20 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_20:
        p_content = div_content_20.find_all('p')
        for li in p_content:
            data_lstma["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_20:
        text_content=div_coord_20.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstma["coordinateur"] = match.group("name").strip()
            data_lstma["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_20:
        prog_content=div_prog_20.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstma["programme"] = semestre_dict

    if div_deb_20:
        deb_content=div_deb_20.get_text().strip()
        #print(deb_content)
        data_lstma["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstma

{'filere': 'LST: Mathématiques et Applications',
 'objectif': 'Acquérir une double compétence mathématiques et informatique grâce à laquelle ils aborderont les problèmes de modélisation et de simulation scientifique sous tous leurs aspects, de la conception à la mise en oeuvre numérique.L’accent est donc mis à la fois sur des techniques d’ingénierie mathématiques et sur les langages de programmation et les outils algorithmiques.',
 'coordinateur': 'Pr.Lahrouz Adil',
 'email': 'a.lahrouz1@uae.ac.ma',
 'debouche': 'Cette licence a pour débouchés :  Les entreprises et les bureaux d’études ; La poursuite des études en cycle ingénieur ; La poursuite des études en cycle Master ; L’enseignement public ou privé.',
 'programme': {'Semestre 5': ['Topologies dans les espaces métriques',
   'Intégration',
   'Probabilités avancées',
   'Recherche Opérationnelle',
   'Concepts de base de programmation orientée C',
   'Analyse numérique matricielle'],
  'Semestre 6': ['Calcul différentiel et équatio

In [132]:
URL_21 = "https://fstt.ac.ma/Portail2023/lst-statistique-et-science-des-donnees/"

data_lstssd={"filere":"LST : Statistique et Science des données",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_21, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_21 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_21 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_21 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_21 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_21:
        p_content = div_content_21.find_all('li')
        for li in p_content:
            data_lstssd["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_21:
        text_content=div_coord_21.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstssd["coordinateur"] = match.group("name").strip()
            data_lstssd["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_21:
        prog_content=div_prog_21.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstssd["programme"] = semestre_dict

    if div_deb_21:
        deb_content=div_deb_21.get_text().strip()
        #print(deb_content)
        data_lstssd["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstssd

{'filere': 'LST : Statistique et Science des données',
 'objectif': 'Permettre aux étudiants d’acquérir des connaissances en mathématiques fondamentales et appliquées et aussi en sciences des données et informatique.Apporter aux étudiants une formation solide en statistique et science des données, tout en leur permettant d’acquérir une culture scientifique générale pour favoriser leur intégration dans leur futurDévelopper les aspects théoriques par des enseignements adaptés (cours, travaux dirigés, étude de cas, mini projets) et des aspects pratiques par des études de problèmes concrets de la statistique, Science des donnée et informatique (stages en entreprise, visites et projets de fin d’études).Utiliser efficacement l’outil informatique et les logicielsFormer des jeunes diplômés, aptes à répondre aux besoins des entreprises dans le domaine du traitement statistique de l’information et des techniques d’aide à la décision.Préparer les étudiants aux carrières de recherche',
 'coordinat

In [134]:
URL_22 = "https://fstt.ac.ma/Portail2023/lst-genie-electrique-option-genieelectrique-systeme-industriel/"

data_lstgsi={"filere":"LST: Génie Electrique Option: GénieElectrique & Système Industriel",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_22, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_22 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_22 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_22 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_22 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_22:
        p_content = div_content_22.find_all('p')
        for li in p_content:
            data_lstgsi["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_22:
        text_content=div_coord_22.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstgsi["coordinateur"] = match.group("name").strip()
            data_lstgsi["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_22:
        prog_content=div_prog_22.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstgsi["programme"] = semestre_dict

    if div_deb_22:
        deb_content=div_deb_22.get_text().strip()
        #print(deb_content)
        data_lstgsi["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstgsi

{'filere': 'LST: Génie Electrique Option: GénieElectrique & Système Industriel',
 'objectif': 'L’objectif de la licence Génie électrique est de donner aux étudiants les éléments de base en physique mathématique et informatique et de leur apporter une formation solide dans les domaines du génie électrique, en particulier en électronique électrotechnique. Ce qui leur permettra de préparer un master à dominante ingénierie EEA ou d’intégrer les grandes écoles d’ingénieurs. Ils pourront aussi se présenter aux différentes fonctions publiques ou privées exigeant le niveau de la licence.Les étudiants sont également initiés à la vie professionnelle par le biais de stages au sein des entreprises.',
 'coordinateur': 'Pr. Mekrini Zineb',
 'email': 'zmekrini@uae.ac.ma',
 'debouche': 'Elles sont complémentaires et transversales à beaucoup de domaines :Électronique de contrôle, de mesure et de commande.Traitement et analyse du signal.Principe de fonctionnement et le domaine d’application des machines

In [136]:
URL_23 = "https://fstt.ac.ma/Portail2023/lst-rrn/"

data_lstrrn={"filere":"LST: Risques et Ressources Naturels",
          "objectif":"",
          "coordinateur":"",
          "email":"",
           "debouche":"",
          "programme":{}
         }

try:
    session = requests.Session()
    r = session.get(URL_23, headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
    r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_content_23 = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
    div_coord_23 = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
    div_prog_23 = soup.find('div',attrs={'data-title-link':"programme-tab"})
    div_deb_23 = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
    
    if div_content_23:
        p_content = div_content_23.find_all('p')
        for li in p_content:
            data_lstrrn["objectif"]+=li.get_text().strip()
            #print(p.get_text().strip())

    if div_coord_23:
        text_content=div_coord_23.get_text().strip()
        #print(text_content)
        # Use regular expressions to extract the name and email
        match = re.search(r"Coordinnateur\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
        if match:
            data_lstrrn["coordinateur"] = match.group("name").strip()
            data_lstrrn["email"] = match.group("email").strip()
        else:
            print("Could not find the expected text format.")

    if div_prog_23:
        prog_content=div_prog_23.get_text().strip()
        #print(prog_content)
        semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
        semestre_dict={}
        # Parcourir les éléments obtenus après la séparation
        for i in range(1, len(semestres), 2):
            semestre_key = semestres[i].strip()  
            points = semestres[i + 1].split('•')  
            points = [point.strip() for point in points if point.strip()]  
            semestre_dict[semestre_key] = points  
        data_lstrrn["programme"] = semestre_dict

    if div_deb_23:
        deb_content=div_deb_23.get_text().strip()
        #print(deb_content)
        data_lstrrn["debouche"]=deb_content.replace("\n"," ")
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")

data_lstrrn

{'filere': 'LST: Risques et Ressources Naturels',
 'objectif': 'Le Maroc est exposé, vu sa position géographique et spécifications géologiques, à la production d’éventuelle catastrophes naturelles, en particulier les séismes, les inondations, les crus torrentielle, le risque hydrique, etc. c’est devenu un sujet de préoccupation en raison de l’impact de ces phénomènes sur le plan socioéconomique. A cet effet, le Maroc a adopté le plan d’action national pour la gestion des risques de catastrophes naturelles couvrant essentiellement 4 types de catastrophes : les tsunamis, l’érosion des sols, les séismes, et les inondations. Sans oublier que la menace de ces catastrophes croit avec le changement climatique et le développement démographique. Ces derniers, influent également sur les réserves des ressources naturelles du pays d’une manière flagrante, les exposant ainsi au risque d’épuisement. En effet, là il s’agit d’un autre risque qui menace le secteur socioéconomique du pays. Ainsi, pour f

In [137]:
collection_deust = db_formation_init['licence_col']
lst_list=[data_lstrrn,data_lstad,data_lstgsi,data_lstssd,data_lstma,data_lstgpro,data_lstginfo,data_lstgi,data_lstgc,data_lster,data_lstdip,data_lstbio]

len(lst_list)

12

In [138]:
for dt in lst_list:
    collection_deust.insert_one(dt)

##### #CI:

In [140]:
links =[
    "https://fstt.ac.ma/Portail2023/genie-electrique-et-managementindustriel/",
    "https://fstt.ac.ma/Portail2023/di-genie-industriel/",
    "https://fstt.ac.ma/Portail2023/di-geoinformation/",
    "https://fstt.ac.ma/Portail2023/di-logiciels-et-systemes-intelligents/"
]

filiers =[
    "DI:Génie Electrique et ManagementIndustriel",
    "DI: Génie Industriel",
    "DI: Géoinformation",
    "DI: Logiciels et Systèmes Intelligents"
]

for i in range(len(links)):
    data_ci={"filere":filiers[i],
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "debouche":"",
          "programme":{}
         }

    try:
        session = requests.Session()
        r = session.get(links[i], headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
        r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        div_content_ci = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
        div_coord_ci = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
        div_prog_ci = soup.find('div',attrs={'data-title-link':"programme-tab"})
        div_deb_ci = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
        
        if div_content_ci:
            p_content = div_content_ci.find_all(['p','li'])
            for li in p_content:
                data_ci["objectif"]+=li.get_text().strip()
                #print(p.get_text().strip())
    
        if div_coord_ci:
            text_content=div_coord_ci.get_text().strip()
            #print(text_content)
            # Use regular expressions to extract the name and email
            match = re.search(r"Coordinnateur pédagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
            if match:
                data_ci["coordinateur"] = match.group("name").strip()
                data_ci["email"] = match.group("email").strip()
            else:
                print("Could not find the expected text format.")
    
        if div_prog_ci:
            prog_content=div_prog_ci.get_text().strip()
            #print(prog_content)
            semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
            semestre_dict={}
            # Parcourir les éléments obtenus après la séparation
            for i in range(1, len(semestres), 2):
                semestre_key = semestres[i].strip()  
                points = semestres[i + 1].split('•')  
                points = [point.strip() for point in points if point.strip()]  
                semestre_dict[semestre_key] = points  
            data_ci["programme"] = semestre_dict
    
        if div_deb_ci:
            deb_content=div_deb_ci.get_text().strip()
            #print(deb_content)
            data_ci["debouche"]=deb_content.replace("\n"," ")
            
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")

    #print(data_ci)
    collection_ci = db_formation_init['ci_col']
    collection_ci.insert_one(data_ci)


##### #MST:

In [147]:
links_ =[
    "https://fstt.ac.ma/Portail2023/mst-intelligence-artificielle-et-sciences-de-donnees/",
    "https://fstt.ac.ma/Portail2023/mst-securite-it-et-big-data/",
    "https://fstt.ac.ma/Portail2023/mst-sciences-delenvironnement/",
    "https://fstt.ac.ma/Portail2023/mst-genie-civil/",
    "https://fstt.ac.ma/Portail2023/mst-modelisation-mathematique-et-science-de-donnees/",
    "https://fstt.ac.ma/Portail2023/mst-sciences-du-littoralapproche-pluridisciplinaire/",
    "https://fstt.ac.ma/Portail2023/mst-sciences-agroalimentaires/",
    "https://fstt.ac.ma/Portail2023/mst-ingenierie-environnementale-changement-climatique-et-developpement-durable/",
    "https://fstt.ac.ma/Portail2023/mst-genie-energetique/",
    "https://fstt.ac.ma/Portail2023/mst-georessources-energetiques-et-reservoirs/",
    "https://fstt.ac.ma/Portail2023/mst-genie-des-materiaux-pour-plasturgie-et-metallurgie/",
    "https://fstt.ac.ma/Portail2023/mst-analyse-appliquee-etingenierie-statistique/",
    "https://fstt.ac.ma/Portail2023/mst-bases-cellulaires-etmoleculaires-en-biotechnologie/"
]

filiers_ =[
    "MST : Intelligence Artificielle et Sciences de Données",
    "MST : Sécurité IT et Big Data",
    "MST : Sciences de l’Environnement",
    "MST : Génie Civil",
    "MST : Modélisation Mathématique et Science de Données",
    "MST : Sciences du Littoral:Approche Pluridisciplinaire",
    "MST : Sciences Agroalimentaires",
    "MST : Ingénierie Environnementale, Changement Climatique et Développement Durable",
    "MST : Génie Energétique",
    "MST : Géoressources Énergétiques et Réservoirs",
    "MST : Génie des Matériaux pour Plasturgie et Métallurgie",
    "MST : Analyse Appliquée etIngénierie Statistique",
    "MST : Bases Cellulaires et Moléculaires en Biotechnologie"
]

for i in range(len(links_)):
    data_mst={"filere":filiers_[i],
          "objectif":"",
          "coordinateur":"",
          "email":"",
          "debouche":"",
          "programme":{}
         }

    try:
        session = requests.Session()
        r = session.get(links_[i], headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
        r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        div_content_mst = soup.find('div', attrs={'data-title-link':"objectifs--tab"})
        div_coord_mst = soup.find('div',attrs={'data-title-link':"coordinateur-tab"})
        div_prog_mst = soup.find('div',attrs={'data-title-link':"programme-tab"})
        div_deb_mst = soup.find('div',attrs={'data-title-link':"competences-visees-et-debouches-tab"})
        
        if div_content_mst:
            p_content = div_content_mst.find_all(['p','li'])
            for li in p_content:
                data_mst["objectif"]+=li.get_text().strip()
                #print(p.get_text().strip())
    
        if div_coord_mst:
            text_content=div_coord_mst.get_text().strip()
            #print(text_content)
            # Use regular expressions to extract the name and email
            match = re.search(r"Coordinnateur [pP]édagogique\s*:\s*(?P<name>[^:]+)\s*:\s*(?P<email>\S+)", text_content)
            if match:
                data_mst["coordinateur"] = match.group("name").strip()
                data_mst["email"] = match.group("email").strip()
            else:
                print("Could not find the expected text format.")
    
        if div_prog_mst:
            prog_content=div_prog_mst.get_text().strip()
            #print(prog_content)
            semestres = re.split(r'(Semestre \d+(?: \(.+?\))?)', prog_content)
            semestre_dict={}
            # Parcourir les éléments obtenus après la séparation
            for i in range(1, len(semestres), 2):
                semestre_key = semestres[i].strip()  
                points = semestres[i + 1].split('•')  
                points = [point.strip() for point in points if point.strip()]  
                semestre_dict[semestre_key] = points  
            data_mst["programme"] = semestre_dict
    
        if div_deb_mst:
            deb_content=div_deb_mst.get_text().strip()
            #print(deb_content)
            data_mst["debouche"]=deb_content.replace("\n"," ")
            
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")

    #print(data_ci)
    collection_mst = db_formation_init['master_col']
    collection_mst.insert_one(data_mst)


##### #Scrape des actualités:

In [149]:
#pip install selenium

In [183]:
URL_actualites = "https://fstt.ac.ma/Portail2023/category/articles/actualites/"

try:
    session = requests.Session()
    r = session.get(URL_actualites, headers=headers, timeout=10)  
    r.raise_for_status() 
    #r.render(sleep=1, scrolldown=5)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    div_gen = soup.find('div',attrs={'class': "elementor-posts-container elementor-posts elementor-posts--skin-classic elementor-grid"})
    actu_list=[]
    articles = div_gen.find_all('article')
    #print(div_gen.prettify())
    for item in articles:
        #print(item.prettify())
        title_item = item.find('h3')
        title = title_item.get_text().strip()
        #print(title)
        a_link=title_item.find('a')    
        link = a_link['href']
        #print(link)
        date_item = item.find('span')
        date= date_item.get_text().strip()
        #print(date)
        actu_={
            'titre':title,
            'lien':link,
            'date':date
        }
        actu_list.append(actu_)
        
    
        
except requests.exceptions.RequestException as e:
    print(f"Une erreur s'est produite : {e}")


In [185]:
#pour les 10 autres pages:
for i in range(1,31):
    URL__actualites = f"https://fstt.ac.ma/Portail2023/category/articles/actualites/page/{i}/"

    try:
        session = requests.Session()
        r = session.get(URL__actualites, headers=headers, timeout=10)  
        r.raise_for_status() 
        #r.render(sleep=1, scrolldown=5)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        div_gen = soup.find('div',attrs={'class': "elementor-posts-container elementor-posts elementor-posts--skin-classic elementor-grid"})
        articles = div_gen.find_all('article')
        #print(div_gen.prettify())
        for item in articles:
            #print(item.prettify())
            title_item = item.find('h3')
            title = title_item.get_text().strip()
            #print(title)
            a_link=title_item.find('a')    
            link = a_link['href']
            #print(link)
            date_item = item.find('span')
            date= date_item.get_text().strip()
            #print(date)
            actu_={
                'titre':title,
                'lien':link,
                'date':date
            }
            actu_list.append(actu_)  
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")

In [186]:
len(actu_list)

310

In [187]:
db_act = client['DATA_FSTT']
collection_act = db_act['actualite_col']
for elem in actu_list:
    collection_act.insert_one(elem)

#### #Scraper les données des équipe de recherches:

In [203]:
links =[
    "https://fstt.ac.ma/Portail2023/greenology/",
    "https://fstt.ac.ma/Portail2023/club-d-art-dramatique-et-d-animation-culturelle/",
    "https://fstt.ac.ma/Portail2023/club-genie-civil/",
    "https://fstt.ac.ma/Portail2023/club-les-sophistes/",
    "https://fstt.ac.ma/Portail2023/club-design-photography/",
    "https://fstt.ac.ma/Portail2023/club-d-astronomie/",
    "https://fstt.ac.ma/Portail2023/quran/",
    "https://fstt.ac.ma/Portail2023/club-echec-mat/",
    "https://fstt.ac.ma/Portail2023/club-enactus/",
    "https://fstt.ac.ma/Portail2023/di-geoinformation/",
    "https://fstt.ac.ma/Portail2023/club-biotechnologie/",
    "https://fstt.ac.ma/Portail2023/club-future-leaders/"
]

clubs =[
    "Greenology",
    "CADAC",
    "Club Génie civil",
    "Club les sophistes",
    "Club Design & Photography",
    "Club d’astronomie",
    "Quran club",
    "Club Echec & Mat",
    "Club Enactus",
    "Géoinformation",
    "Club Biotechnologie",
    "Future Leaders"
    
]

for i in range(len(links)):
    data_c={"club":clubs[i],"informations":""}

    try:
        session = requests.Session()
        r = session.get(links[i], headers=headers, timeout=10)  # Ajouter un délai (timeout) de 10 secondes
        r.raise_for_status()  # Vérifier si la requête a réussi (renvoie une exception pour les statuts d'erreur HTTP)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        div_content_ci = soup.find('div', attrs={'data-id':"faf7450"})
        
        if div_content_ci:
            p_content = div_content_ci.find_all(['p','h3'])
            for li in p_content:
                data_c["informations"]+=li.get_text(separator=",").strip()
                #print(li.get_text().strip())
            
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")

    #print(data_ci)
    collection_c = db['club_col']
    collection_c.insert_one(data_c)


Le Club de l’environnement de la FST de Tanger, Dénommé Greenology Club, est nouvellement créé et il a comme vocation de : – Transmettre aux étudiants, une conscience nationale éclairée, un patriotisme activé pour un environnement préservé. – Sensibiliser et rendre les étudiants de la FST de Tanger des combattants contre l’éparpillement de l’environnement. – Tous unis, bien convaincus que nos richesses environnementales très précieuses sont menacées, et qu’avec une conscience large du désastre écologique qui frappe à la porte de notre pays, on pourra gagner la bataille et sauver ce qu’il reste encore à sauver.
la page facebook:
https://www.facebook.com/greenology.club
Email de contact :
Responsable d’activités 4saaddine@gmail.com
https://www.facebook.com/CADAC.FSTT
Le Club Génie civil est un club scientifique de la faculté des sciences et techniques Tanger crée en 2014 par l’étudiant KALOUCH Hammam Qui vise a développer la culture scientifiques chez les étudiants de la FSTT ainsi que p